# Run PPO on Ant from [here](https://github.com/pat-coady/trpo)
#### More descriptions can be found on Patrick Coady's blog regarding [gym and ppo](https://learningai.io/projects/2017/07/28/ai-gym-workout.html) or [descriptions about Ant env](https://gist.github.com/pat-coady/bac60888f011199aad72d2f1e6f5a4fa)

In [1]:
import gym,mujoco_py,warnings,time,os,glob,shutil,csv,skvideo.io
gym.logger.set_level(40)
warnings.filterwarnings("ignore") 
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from gym.envs import mujoco
from datetime import datetime
from util import PID_class,Scaler,Logger,display_frames_as_gif
from custom_ant import AntEnvCustom
from ppo import NNValueFunction,Policy,run_episode,run_policy,add_value,discount,\
    add_disc_sum_rew,add_gae,build_train_set,log_batch_stats,run_episode_vid
np.set_printoptions(precision=2,linewidth=150)
%matplotlib inline  
%config InlineBackend.figure_format = 'retina'
print ("Packages Loaded") 

Packages Loaded


### main

In [2]:
env = AntEnvCustom()
obs_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]
env.reset() # Reset 
# render_img = env.render(mode='rgb_array')
print ("obs_dim:[%d] act_dim:[%d]"%(obs_dim,act_dim))

obs_dim += 1  # add 1 to obs dimension for time step feature (see run_episode())
# Logger
env_name = 'Ant'
now = datetime.utcnow().strftime("%b-%d_%H:%M:%S")  # create unique directories
logger = Logger(logName=env_name,now=now,_NOTUSE=True)
aigym_path = os.path.join('/tmp', env_name, now)
# Scaler
scaler = Scaler(obs_dim)
# Value function
hid1_mult = 10
val_func = NNValueFunction(obs_dim, hid1_mult)
# Policy Function
kl_targ = 0.003
policy_logvar = -1.0
policy = Policy(obs_dim, act_dim, kl_targ, hid1_mult, policy_logvar)

Custom Ant Environment made by SJ.
obs_dim:[111] act_dim:[8]
Value Params -- h1: 1120, h2: 74, h3: 5, lr: 0.00116
Policy Params -- h1: 1120, h2: 299, h3: 80, lr: 5.2e-05, logvar_speed: 16
setting up loss with KL penalty


### Run policy for the first time

In [3]:
trajectories = run_policy(env, policy, scaler, logger, episodes=5)
add_value(trajectories, val_func)  # add estimated values to episodes
gamma = 0.995 # Discount factor 
lam = 0.95 # Lambda for GAE
add_disc_sum_rew(trajectories, gamma)  # calculated discounted sum of Rs
add_gae(trajectories, gamma, lam)  # calculate advantage
print ('observes shape:',trajectories[0]['observes'].shape)
print ('actions shape:',trajectories[0]['actions'].shape)
print ('rewards shape:',trajectories[0]['rewards'].shape)
print ('unscaled_obs shape:',trajectories[0]['unscaled_obs'].shape)
print ('values shape:',trajectories[0]['values'].shape)
print ('disc_sum_rew shape:',trajectories[0]['disc_sum_rew'].shape)
print ('advantages shape:',trajectories[0]['advantages'].shape)

observes shape: (135, 112)
actions shape: (135, 8)
rewards shape: (135,)
unscaled_obs shape: (135, 112)
values shape: (135,)
disc_sum_rew shape: (135,)
advantages shape: (135,)


### Loop

In [4]:
SAVE_VID = True
MAKE_GIF = False 

In [5]:
maxEpoch  = 10000
batchSize = 100
for _epoch in range(maxEpoch):
    # 1. Run policy
    trajectories = run_policy(env, policy, scaler, logger, episodes=batchSize)
    # 2. Get (predict) value from the critic network 
    add_value(trajectories, val_func)  # add estimated values to episodes
    # 3. Get GAE
    gamma = 0.995 # Discount factor 
    lam = 0.95 # Lambda for GAE
    add_disc_sum_rew(trajectories, gamma)  # calculated discounted sum of Rs
    add_gae(trajectories, gamma, lam)  # calculate advantage
    # concatenate all episodes into single NumPy arrays
    observes, actions, advantages, disc_sum_rew = build_train_set(trajectories)
    # add various stats to training log:
    # log_batch_stats(observes, actions, advantages, disc_sum_rew, logger, episode)
    # Update
    policy.update(observes, actions, advantages, logger)  # update policy
    val_func.fit(observes, disc_sum_rew, logger)  # update value function
    # logger.write(display=True)  # write logger results to file and stdout
    
    # Print
    for _tIdx in range(len(trajectories)):
        rs = trajectories[_tIdx]['rewards']
        if _tIdx == 0: rTotal = rs
        else: rTotal = np.concatenate((rTotal,rs))
        # Reward details      
    avgRwd = rTotal.mean()
    reward_contacts,reward_ctrls,reward_forwards,reward_headings,reward_survives = [],[],[],[],[]
    for traj in trajectories:
        cTraj = traj['rDetails']
        for _iIdx in range(len(cTraj)):
            reward_contacts.append(cTraj[_iIdx]['reward_contact'])
            reward_ctrls.append(cTraj[_iIdx]['reward_ctrl'])
            reward_forwards.append(cTraj[_iIdx]['reward_forward'])
            reward_headings.append(cTraj[_iIdx]['reward_heading'])
            reward_survives.append(cTraj[_iIdx]['reward_survive'])
    avgReward_contact = np.asarray(reward_contacts).mean()
    avgReward_ctrl = np.asarray(reward_ctrls).mean()
    avgReward_forward = np.asarray(reward_forwards).mean()
    avgReward_heading = np.asarray(reward_headings).mean()
    avgReward_survive = np.asarray(reward_survives).mean()
    print ("[%d/%d](#total:%d) avgRwd:[%.3f](cntct:%.3f+ctrl:%.3f+fwd:%.3f+head:%.3f+srv:%.3f)"%
           (_epoch,maxEpoch,(_epoch+1)*batchSize,avgRwd,
           avgReward_contact,avgReward_ctrl,avgReward_forward,avgReward_heading,avgReward_survive))
    
    # SHOW EVERY 
    PLOT_EVERY = 20
    DO_ANIMATE = False
    if ((_epoch%PLOT_EVERY)==0 ) | (_epoch==(maxEpoch-1)):
        ret = run_episode_vid(env, policy, scaler)
        print ("  [^] avgRwd:[%.3f] Xdisp:[%.3f] hDisp:[%.1f]"%
               (np.asarray(ret['rewards']).mean(),ret['xDisp'],ret['hDisp']))
        if MAKE_GIF:
            display_frames_as_gif(ret['frames'])
        if SAVE_VID:
            outputdata = np.asarray(ret['frames']).astype(np.uint8)
            vidName = 'vids/ant_ppo_epoch%03d.mp4'%(_epoch)
            skvideo.io.vwrite(vidName,outputdata)
            print ("[%s] saved."%(vidName))
print ("Done.")

[0/10000](#total:100) avgRwd:[-3.827](cntct:-0.001+ctrl:-1.932+fwd:-0.010+head:-2.884+srv:1.000)
Creating window glfw
  [^] avgRwd:[-2.165] Xdisp:[-2.682] hDisp:[-31.1]
[vids/ant_ppo_epoch000.mp4] saved.
[1/10000](#total:200) avgRwd:[-3.640](cntct:-0.001+ctrl:-1.992+fwd:-0.054+head:-2.593+srv:1.000)
[2/10000](#total:300) avgRwd:[-4.026](cntct:-0.001+ctrl:-1.962+fwd:-0.016+head:-3.047+srv:1.000)
[3/10000](#total:400) avgRwd:[-4.280](cntct:-0.001+ctrl:-1.975+fwd:-0.012+head:-3.292+srv:1.000)
[4/10000](#total:500) avgRwd:[-4.598](cntct:-0.001+ctrl:-1.950+fwd:-0.022+head:-3.625+srv:1.000)
[5/10000](#total:600) avgRwd:[-3.493](cntct:-0.001+ctrl:-1.970+fwd:-0.015+head:-2.506+srv:1.000)
[6/10000](#total:700) avgRwd:[-3.629](cntct:-0.001+ctrl:-1.972+fwd:0.023+head:-2.679+srv:1.000)
[7/10000](#total:800) avgRwd:[-3.953](cntct:-0.001+ctrl:-2.018+fwd:0.037+head:-2.971+srv:1.000)
[8/10000](#total:900) avgRwd:[-4.132](cntct:-0.001+ctrl:-2.001+fwd:0.004+head:-3.134+srv:1.000)
[9/10000](#total:1000) 

[80/10000](#total:8100) avgRwd:[-2.748](cntct:-0.001+ctrl:-2.109+fwd:0.024+head:-1.662+srv:1.000)
  [^] avgRwd:[-8.586] Xdisp:[-0.312] hDisp:[66.9]
[vids/ant_ppo_epoch080.mp4] saved.
[81/10000](#total:8200) avgRwd:[-3.540](cntct:-0.001+ctrl:-2.121+fwd:0.029+head:-2.447+srv:1.000)
[82/10000](#total:8300) avgRwd:[-2.870](cntct:-0.001+ctrl:-2.153+fwd:0.041+head:-1.758+srv:1.000)
[83/10000](#total:8400) avgRwd:[-3.128](cntct:-0.001+ctrl:-2.139+fwd:0.050+head:-2.038+srv:1.000)
[84/10000](#total:8500) avgRwd:[-2.646](cntct:-0.001+ctrl:-2.095+fwd:0.095+head:-1.645+srv:1.000)
[85/10000](#total:8600) avgRwd:[-3.416](cntct:-0.001+ctrl:-2.100+fwd:0.024+head:-2.339+srv:1.000)
[86/10000](#total:8700) avgRwd:[-3.496](cntct:-0.001+ctrl:-2.121+fwd:-0.007+head:-2.367+srv:1.000)
[87/10000](#total:8800) avgRwd:[-3.317](cntct:-0.001+ctrl:-2.129+fwd:0.020+head:-2.207+srv:1.000)
[88/10000](#total:8900) avgRwd:[-3.553](cntct:-0.001+ctrl:-2.200+fwd:0.039+head:-2.390+srv:1.000)
[89/10000](#total:9000) avgRwd:[

[159/10000](#total:16000) avgRwd:[-2.202](cntct:-0.001+ctrl:-2.292+fwd:0.147+head:-1.056+srv:1.000)
[160/10000](#total:16100) avgRwd:[-3.562](cntct:-0.001+ctrl:-2.260+fwd:0.082+head:-2.384+srv:1.000)
  [^] avgRwd:[-4.567] Xdisp:[6.963] hDisp:[86.2]
[vids/ant_ppo_epoch160.mp4] saved.
[161/10000](#total:16200) avgRwd:[-3.681](cntct:-0.001+ctrl:-2.307+fwd:-0.007+head:-2.366+srv:1.000)
[162/10000](#total:16300) avgRwd:[-3.091](cntct:-0.001+ctrl:-2.342+fwd:0.123+head:-1.872+srv:1.000)
[163/10000](#total:16400) avgRwd:[-3.838](cntct:-0.001+ctrl:-2.400+fwd:0.131+head:-2.568+srv:1.000)
[164/10000](#total:16500) avgRwd:[-3.255](cntct:-0.001+ctrl:-2.384+fwd:0.010+head:-1.880+srv:1.000)
[165/10000](#total:16600) avgRwd:[-2.758](cntct:-0.001+ctrl:-2.333+fwd:0.055+head:-1.479+srv:1.000)
[166/10000](#total:16700) avgRwd:[-3.221](cntct:-0.001+ctrl:-2.427+fwd:0.048+head:-1.842+srv:1.000)
[167/10000](#total:16800) avgRwd:[-3.421](cntct:-0.001+ctrl:-2.389+fwd:0.062+head:-2.092+srv:1.000)
[168/10000](#to

[238/10000](#total:23900) avgRwd:[-2.995](cntct:-0.001+ctrl:-2.540+fwd:0.117+head:-1.571+srv:1.000)
[239/10000](#total:24000) avgRwd:[-3.331](cntct:-0.001+ctrl:-2.539+fwd:-0.014+head:-1.778+srv:1.000)
[240/10000](#total:24100) avgRwd:[-3.767](cntct:-0.001+ctrl:-2.541+fwd:0.100+head:-2.324+srv:1.000)
  [^] avgRwd:[-10.711] Xdisp:[-1.228] hDisp:[160.6]
[vids/ant_ppo_epoch240.mp4] saved.
[241/10000](#total:24200) avgRwd:[-3.564](cntct:-0.001+ctrl:-2.643+fwd:0.107+head:-2.027+srv:1.000)
[242/10000](#total:24300) avgRwd:[-3.366](cntct:-0.001+ctrl:-2.648+fwd:0.118+head:-1.835+srv:1.000)
[243/10000](#total:24400) avgRwd:[-3.391](cntct:-0.001+ctrl:-2.529+fwd:0.079+head:-1.940+srv:1.000)
[244/10000](#total:24500) avgRwd:[-3.547](cntct:-0.001+ctrl:-2.548+fwd:0.002+head:-2.000+srv:1.000)
[245/10000](#total:24600) avgRwd:[-2.242](cntct:-0.001+ctrl:-2.478+fwd:0.169+head:-0.932+srv:1.000)
[246/10000](#total:24700) avgRwd:[-2.725](cntct:-0.001+ctrl:-2.498+fwd:0.011+head:-1.237+srv:1.000)
[247/10000](

[317/10000](#total:31800) avgRwd:[-3.492](cntct:-0.001+ctrl:-2.827+fwd:0.015+head:-1.680+srv:1.000)
[318/10000](#total:31900) avgRwd:[-2.802](cntct:-0.001+ctrl:-2.648+fwd:0.121+head:-1.275+srv:1.000)
[319/10000](#total:32000) avgRwd:[-3.515](cntct:-0.001+ctrl:-2.718+fwd:-0.030+head:-1.767+srv:1.000)
[320/10000](#total:32100) avgRwd:[-3.205](cntct:-0.001+ctrl:-2.711+fwd:0.055+head:-1.549+srv:1.000)
  [^] avgRwd:[-6.455] Xdisp:[0.794] hDisp:[104.3]
[vids/ant_ppo_epoch320.mp4] saved.
[321/10000](#total:32200) avgRwd:[-3.480](cntct:-0.001+ctrl:-2.616+fwd:0.109+head:-1.972+srv:1.000)
[322/10000](#total:32300) avgRwd:[-2.850](cntct:-0.001+ctrl:-2.705+fwd:0.111+head:-1.255+srv:1.000)
[323/10000](#total:32400) avgRwd:[-2.783](cntct:-0.001+ctrl:-2.651+fwd:0.065+head:-1.196+srv:1.000)
[324/10000](#total:32500) avgRwd:[-3.346](cntct:-0.001+ctrl:-2.697+fwd:0.086+head:-1.735+srv:1.000)
[325/10000](#total:32600) avgRwd:[-3.504](cntct:-0.001+ctrl:-2.786+fwd:0.100+head:-1.816+srv:1.000)
[326/10000](#t

...

[9677/10000](#total:967800) avgRwd:[0.833](cntct:-0.001+ctrl:-0.733+fwd:0.595+head:-0.028+srv:1.000)
[9678/10000](#total:967900) avgRwd:[0.493](cntct:-0.001+ctrl:-0.809+fwd:0.491+head:-0.188+srv:1.000)
[9679/10000](#total:968000) avgRwd:[0.735](cntct:-0.001+ctrl:-0.730+fwd:0.575+head:-0.108+srv:1.000)
[9680/10000](#total:968100) avgRwd:[-1.819](cntct:-0.001+ctrl:-1.420+fwd:0.516+head:-1.914+srv:1.000)
  [^] avgRwd:[-12.455] Xdisp:[0.768] hDisp:[50.5]
[vids/ant_ppo_epoch9680.mp4] saved.
[9681/10000](#total:968200) avgRwd:[0.968](cntct:-0.001+ctrl:-0.601+fwd:0.602+head:-0.033+srv:1.000)
[9682/10000](#total:968300) avgRwd:[0.930](cntct:-0.001+ctrl:-0.651+fwd:0.606+head:-0.024+srv:1.000)
[9683/10000](#total:968400) avgRwd:[0.751](cntct:-0.001+ctrl:-0.699+fwd:0.558+head:-0.107+srv:1.000)
[9684/10000](#total:968500) avgRwd:[0.406](cntct:-0.001+ctrl:-0.866+fwd:0.587+head:-0.314+srv:1.000)
[9685/10000](#total:968600) avgRwd:[0.817](cntct:-0.001+ctrl:-0.710+fwd:0.570+head:-0.042+srv:1.000)
[968

[9755/10000](#total:975600) avgRwd:[0.875](cntct:-0.001+ctrl:-0.686+fwd:0.605+head:-0.043+srv:1.000)
[9756/10000](#total:975700) avgRwd:[0.841](cntct:-0.001+ctrl:-0.707+fwd:0.587+head:-0.039+srv:1.000)
[9757/10000](#total:975800) avgRwd:[0.812](cntct:-0.001+ctrl:-0.747+fwd:0.592+head:-0.032+srv:1.000)
[9758/10000](#total:975900) avgRwd:[0.952](cntct:-0.001+ctrl:-0.605+fwd:0.580+head:-0.022+srv:1.000)
[9759/10000](#total:976000) avgRwd:[0.604](cntct:-0.001+ctrl:-0.925+fwd:0.601+head:-0.071+srv:1.000)
[9760/10000](#total:976100) avgRwd:[0.883](cntct:-0.001+ctrl:-0.669+fwd:0.584+head:-0.030+srv:1.000)
  [^] avgRwd:[-7.021] Xdisp:[1.652] hDisp:[-157.6]
[vids/ant_ppo_epoch9760.mp4] saved.
[9761/10000](#total:976200) avgRwd:[0.447](cntct:-0.001+ctrl:-0.910+fwd:0.548+head:-0.189+srv:1.000)
[9762/10000](#total:976300) avgRwd:[0.922](cntct:-0.001+ctrl:-0.661+fwd:0.610+head:-0.027+srv:1.000)
[9763/10000](#total:976400) avgRwd:[1.006](cntct:-0.001+ctrl:-0.560+fwd:0.587+head:-0.020+srv:1.000)
[976

[9833/10000](#total:983400) avgRwd:[0.876](cntct:-0.001+ctrl:-0.692+fwd:0.619+head:-0.050+srv:1.000)
[9834/10000](#total:983500) avgRwd:[0.720](cntct:-0.001+ctrl:-0.735+fwd:0.582+head:-0.126+srv:1.000)
[9835/10000](#total:983600) avgRwd:[0.692](cntct:-0.001+ctrl:-0.838+fwd:0.599+head:-0.068+srv:1.000)
[9836/10000](#total:983700) avgRwd:[0.842](cntct:-0.001+ctrl:-0.679+fwd:0.617+head:-0.095+srv:1.000)
[9837/10000](#total:983800) avgRwd:[0.856](cntct:-0.001+ctrl:-0.705+fwd:0.609+head:-0.048+srv:1.000)
[9838/10000](#total:983900) avgRwd:[-0.877](cntct:-0.001+ctrl:-1.357+fwd:0.525+head:-1.044+srv:1.000)
[9839/10000](#total:984000) avgRwd:[0.848](cntct:-0.001+ctrl:-0.724+fwd:0.601+head:-0.028+srv:1.000)
[9840/10000](#total:984100) avgRwd:[1.014](cntct:-0.001+ctrl:-0.555+fwd:0.591+head:-0.022+srv:1.000)
  [^] avgRwd:[-3.899] Xdisp:[1.575] hDisp:[15.6]
[vids/ant_ppo_epoch9840.mp4] saved.
[9841/10000](#total:984200) avgRwd:[0.748](cntct:-0.001+ctrl:-0.783+fwd:0.588+head:-0.056+srv:1.000)
[9842

[9911/10000](#total:991200) avgRwd:[0.974](cntct:-0.001+ctrl:-0.596+fwd:0.593+head:-0.022+srv:1.000)
[9912/10000](#total:991300) avgRwd:[0.980](cntct:-0.001+ctrl:-0.588+fwd:0.590+head:-0.022+srv:1.000)
[9913/10000](#total:991400) avgRwd:[0.839](cntct:-0.001+ctrl:-0.717+fwd:0.579+head:-0.023+srv:1.000)
[9914/10000](#total:991500) avgRwd:[0.848](cntct:-0.001+ctrl:-0.689+fwd:0.582+head:-0.044+srv:1.000)
[9915/10000](#total:991600) avgRwd:[1.002](cntct:-0.001+ctrl:-0.569+fwd:0.593+head:-0.021+srv:1.000)
[9916/10000](#total:991700) avgRwd:[1.023](cntct:-0.001+ctrl:-0.560+fwd:0.605+head:-0.020+srv:1.000)
[9917/10000](#total:991800) avgRwd:[0.935](cntct:-0.001+ctrl:-0.621+fwd:0.593+head:-0.036+srv:1.000)
[9918/10000](#total:991900) avgRwd:[0.876](cntct:-0.001+ctrl:-0.680+fwd:0.613+head:-0.056+srv:1.000)
[9919/10000](#total:992000) avgRwd:[0.645](cntct:-0.001+ctrl:-0.850+fwd:0.566+head:-0.071+srv:1.000)
[9920/10000](#total:992100) avgRwd:[0.469](cntct:-0.001+ctrl:-0.786+fwd:0.570+head:-0.314+s

[9989/10000](#total:999000) avgRwd:[-0.733](cntct:-0.001+ctrl:-1.270+fwd:0.497+head:-0.960+srv:1.000)
[9990/10000](#total:999100) avgRwd:[1.012](cntct:-0.001+ctrl:-0.560+fwd:0.593+head:-0.021+srv:1.000)
[9991/10000](#total:999200) avgRwd:[0.957](cntct:-0.001+ctrl:-0.609+fwd:0.587+head:-0.020+srv:1.000)
[9992/10000](#total:999300) avgRwd:[0.839](cntct:-0.001+ctrl:-0.665+fwd:0.568+head:-0.063+srv:1.000)
[9993/10000](#total:999400) avgRwd:[0.977](cntct:-0.001+ctrl:-0.590+fwd:0.607+head:-0.039+srv:1.000)
[9994/10000](#total:999500) avgRwd:[0.959](cntct:-0.001+ctrl:-0.618+fwd:0.602+head:-0.025+srv:1.000)
[9995/10000](#total:999600) avgRwd:[0.855](cntct:-0.001+ctrl:-0.717+fwd:0.603+head:-0.030+srv:1.000)
[9996/10000](#total:999700) avgRwd:[0.880](cntct:-0.001+ctrl:-0.666+fwd:0.606+head:-0.059+srv:1.000)
[9997/10000](#total:999800) avgRwd:[0.795](cntct:-0.001+ctrl:-0.758+fwd:0.626+head:-0.073+srv:1.000)
[9998/10000](#total:999900) avgRwd:[-1.691](cntct:-0.001+ctrl:-1.420+fwd:0.511+head:-1.781

### Animate final motion

In [6]:
SAVE_VID_FINAL = True
MAKE_GIF_FINAL = False 

In [7]:
for _i in range(3):
    ret = run_episode_vid(env, policy, scaler)
    if MAKE_GIF_FINAL:
        display_frames_as_gif(ret['frames'])
    if SAVE_VID_FINAL:
        outputdata = np.asarray(ret['frames']).astype(np.uint8)
        vidName = 'vids/ant_ppo_final_%d.mp4'%(_i)
        skvideo.io.vwrite(vidName,outputdata)
        print ("[%s] saved."%(vidName))

[vids/ant_ppo_final_0.mp4] saved.
[vids/ant_ppo_final_1.mp4] saved.
[vids/ant_ppo_final_2.mp4] saved.


### Finished

In [8]:
DO_CLOSE = False # There is no turning back. 
if DO_CLOSE:
    logger.close()
    policy.close_sess()
    val_func.close_sess()